In [2]:
!pip install transformers
!pip install pyctcdecode
!apt-get install ffmpeg
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install jsonlines
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.0/458.0 kB 26.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.6/553.6 kB 5.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=592714 sha256=25925533f09ba1bdf60930f9ba913d774da54f69bd1e9fca8e6e6ce5db84d2c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-yfsr25zc/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
import nltk
import librosa
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM

import IPython.display as ipd
import torch.nn as nn
import jsonlines

from transformers import AutoTokenizer
from pyctcdecode import build_ctcdecoder

In [4]:
model_name = 'Yehor/wav2vec2-xls-r-base-uk-with-small-lm'
sampling_rate = 16_000
device = 'cuda' # or cpu

# Dataset

In [16]:
audio_input, sample_rate_original = librosa.load('/kaggle/input/toronto-dataset/toronto_25/toronto_25_278.wav', sr=sampling_rate)
print(f"Sample rate: {sample_rate_original}")

Sample rate: 16000


In [5]:
ipd.Audio(audio_input, rate=sampling_rate)

In [44]:
with jsonlines.open("/kaggle/input/toronto-dataset/labels.jsonl", 'r') as reader:
    for line in reader:
        labels = line

In [7]:
len(labels)

29232

In [8]:
labels["dataset/toronto_25/toronto_25_278.wav"]

'А перший і єдиний кліп «У мене немає дому» – майже 10 мільйонів. Хей, люди, такі люди.'

# Wav2Vec2Processor

In [40]:
processor = Wav2Vec2Processor.from_pretrained(model_name)
# you also might init processor as the following:
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv

In [42]:
logits

tensor([[[ -3.0565,  -1.1659,  -4.1894,  ...,  10.3348, -23.9120, -24.0170],
         [ -2.2030,  -1.6248,  -2.7688,  ...,   9.4862, -23.2198, -23.1169],
         [ -0.8111,  -0.1290,  -2.1774,  ...,  10.2483, -23.9999, -23.8950],
         ...,
         [ -1.1206,   1.7362,  -6.4825,  ...,   7.7800, -21.8098, -21.8994],
         [ -0.2209,  -3.1262,  -3.8794,  ...,   9.9110, -24.4830, -24.7070],
         [  2.2578,  -2.7421,  -3.2106,  ...,   9.6900, -23.8200, -24.1223]]],
       device='cuda:0')

In [41]:
features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
    logits = model(input_values, attention_mask=attention_mask).logits
    
print(logits.shape)
predicted_ids = torch.argmax(logits, dim=-1)
predicted_ids

torch.Size([1, 330, 39])


tensor([[36, 36, 36, 36,  1, 36,  1, 36, 36, 36, 36, 36,  0,  0, 36, 16, 36, 36,
         36, 36, 36,  6, 36, 36, 17, 36, 36, 36, 36, 25, 36, 36,  9, 36, 36, 10,
         36, 36,  0,  0, 36, 36, 31, 36,  0, 36, 36, 36, 30, 36, 36, 36, 36,  5,
         36,  9, 36, 36, 36, 14, 36, 36,  9, 36, 10,  0, 36, 36, 11, 36, 36, 12,
         36, 36, 31, 36, 36, 36, 16, 36, 36,  0,  0, 36, 20, 36, 36,  0, 36, 13,
         36, 36,  6, 36, 36, 36, 14, 36,  6,  0,  0, 14, 36,  6, 36,  0, 36, 13,
         36, 36, 36,  1, 36, 36, 36, 36, 30, 36, 36,  0, 36,  5, 36, 36, 15, 36,
         36, 13, 36, 36, 20, 36,  0, 36, 13, 36, 36, 36,  1, 36, 36, 36, 36, 10,
         36, 36, 36, 36,  7, 36, 36,  6, 36, 36, 36, 36, 36,  0,  0, 36, 36,  5,
         36, 36, 36, 36,  6, 36, 36, 36, 36, 36, 36, 18, 36, 36, 29, 36, 19, 27,
         27,  0, 13, 13, 36, 31, 31, 12, 36, 27, 27, 10, 10, 36, 36, 15, 36, 36,
         14, 14, 36, 31, 36,  3,  3, 36, 36, 36, 36, 36, 36, 36, 36,  0, 36, 36,
         36, 36, 36, 36, 36,

In [12]:
# unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
" ".join(processor.tokenizer.convert_ids_to_tokens(predicted_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] а [PAD] а [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] п [PAD] [PAD] [PAD] [PAD] [PAD] е [PAD] [PAD] р [PAD] [PAD] [PAD] [PAD] ш [PAD] [PAD] и [PAD] [PAD] й [PAD] [PAD] | | [PAD] [PAD] і [PAD] | [PAD] [PAD] [PAD] є [PAD] [PAD] [PAD] [PAD] д [PAD] и [PAD] [PAD] [PAD] н [PAD] [PAD] и [PAD] й | [PAD] [PAD] к [PAD] [PAD] л [PAD] [PAD] і [PAD] [PAD] [PAD] п [PAD] [PAD] | | [PAD] у [PAD] [PAD] | [PAD] м [PAD] [PAD] е [PAD] [PAD] [PAD] н [PAD] е | | н [PAD] е [PAD] | [PAD] м [PAD] [PAD] [PAD] а [PAD] [PAD] [PAD] [PAD] є [PAD] [PAD] | [PAD] д [PAD] [PAD] о [PAD] [PAD] м [PAD] [PAD] у [PAD] | [PAD] м [PAD] [PAD] [PAD] а [PAD] [PAD] [PAD] [PAD] й [PAD] [PAD] [PAD] [PAD] ж [PAD] [PAD] е [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] д [PAD] [PAD] [PAD] [PAD] е [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] с [PAD] [PAD] я [PAD] т ь ь | м м [PAD] і і л [PAD] ь ь й й [PAD] [PAD] о [PAD] [PAD] н н [PAD] і [PAD] в в [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] [PAD] [PAD] [PAD] [PA

In [13]:
transcription = processor.batch_decode(predicted_ids)
transcription

['аа перший і єдиний кліп у мене не має дому майже десять мільйонів кілю']

# Processor with LM

In [14]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

alphabet.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

language_model/attrs.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

language_model/5gram_correct.arpa:   0%|          | 0.00/28.3M [00:00<?, ?B/s]

language_model/unigrams.txt:   0%|          | 0.00/684k [00:00<?, ?B/s]

Loading the LM will be faster if you build a binary file.
Reading /root/.cache/pyctcdecode/models--Yehor--wav2vec2-xls-r-base-uk-with-small-lm/snapshots/b958c7f8aecb6ad9de8702c2ec63462a3d02b62c/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [15]:
features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
    logits = model(input_values, attention_mask=attention_mask).logits

predictions = processor.batch_decode(logits.cpu().numpy()).text
predictions

['а перший і єдиний кліпу мене не має дому майже десять мільйонів к']

# CTC loss

In [45]:
target_transcription = labels["dataset/toronto_25/toronto_25_278.wav"]


# When used in normal mode, this method forwards all its arguments to
# Wav2Vec2FeatureExtractor’s pad() and returns its output. If used in
# the context as_target_processor() this method forwards all its arguments
# to PreTrainedTokenizer’s pad().

with processor.as_target_processor():
    labels = processor(target_transcription, return_tensors="pt").input_ids

loss = model(input_values, labels=labels).loss
loss.backward()

print(loss)

tensor(3.4114, device='cuda:0', grad_fn=<MeanBackward0>)


In [46]:
log_probs = nn.functional.log_softmax(logits, dim=-1, dtype=torch.float32).transpose(0, 1)
log_probs.shape

torch.Size([330, 1, 39])

In [47]:
labels_mask = labels >= 0
target_lengths = labels_mask.sum(-1)
flattened_targets = labels.masked_select(labels_mask)

In [31]:
attention_mask = (
                attention_mask if attention_mask is not None else torch.ones_like(input_values, dtype=torch.long)
           )
#  Computes the output length of the convolutional layers
input_lengths = model._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)

In [32]:
attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0', dtype=torch.int32)

In [34]:
ctc_loss = nn.CTCLoss()
loss = nn.functional.ctc_loss(log_probs, flattened_targets, input_lengths, target_lengths, reduction="mean", blank=36)
loss

tensor(3.4114, device='cuda:0')

# Homework

In [ ]:
# TODO: implement class trainer to fine tune Wav2Vec with CTC on toronto dataset. You may refer to the HF Trainer, but you have to implement it on your own.
# Don't forget about text normalization
# Divide dataset into train, eval and test. Compare performace on test set using original model, original model with original LM, fine-tuned model, fine-tuned model with new LM
# Link to the Toronto dataset - https://drive.google.com/file/d/1j9d91QqE7_WnOnmEmidtOG55tpmxQUeJ/view?usp=sharing
# Another training dataset https://huggingface.co/datasets/skypro1111/whisper-dataset-ytb-uk (I'm not sure about its quality)
# Link to available ASR resources in the Ukrainian language https://github.com/egorsmkv/speech-recognition-uk

## Train/val/test split

In [8]:
with jsonlines.open("/kaggle/input/toronto-dataset/labels.jsonl", 'r') as reader:
    dataset = {path: label for line in reader for path, label in line.items() if os.path.exists("/kaggle/input/toronto-dataset/"+"/".join(path.split("/")[1:])) and label != ""}

len(dataset)

18210

In [22]:
import random
random_sample = random.sample(dataset.items(), k=100)
random_sample_dict = dict(random_sample)

len(random_sample_dict)

/tmp/ipykernel_34/3360363897.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_sample = random.sample(dataset.items(), k=100)


100

In [24]:
from sklearn.model_selection import train_test_split

paths = list(random_sample_dict.keys())
labels = list(random_sample_dict.values())

train_paths, temp_paths, train_labels, temp_labels = train_test_split(paths, labels, test_size=0.2, random_state=42)

validation_paths, test_paths, validation_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)

train_set = {path: label for path, label in zip(train_paths, train_labels)}
validation_set = {path: label for path, label in zip(validation_paths, validation_labels)}
test_set = {path: label for path, label in zip(test_paths, test_labels)}

print("Number of samples in train set:", len(train_set))
print("Number of samples in validation set:", len(validation_set))
print("Number of samples in test set:", len(test_set))

Number of samples in train set: 80
Number of samples in validation set: 10
Number of samples in test set: 10


## Custom dataset

In [46]:
import re
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, dataset, processor, sr):
        self.paths = list(dataset.keys())
        self.labels = list(dataset.values())
        self.processor = processor
        self.sr = sr

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        target_transcription = self.labels[idx]
        target_transcription = self.normalize(target_transcription)
        
        audio_input, _ = librosa.load("/kaggle/input/toronto-dataset/"+"/".join(path.split("/")[1:]), sr=self.sr)
        features = self.processor([audio_input], sampling_rate=self.sr, padding=True, return_tensors='pt')
    
        labels = self.processor(text=target_transcription, return_tensors="pt").input_ids
        return features, labels, target_transcription
    
    def normalize(self, text):
        text = text.lower()
        text = re.sub(r'\[[^\[\]]*\]', "", text)
        text = re.sub(r'\([^()]*\)', "", text)
        text = re.sub("\s+", " ", text)
        return text.strip()

## Collate func

In [44]:
asr_model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [45]:
dataset_hehe = CustomDataset(test_set, processor, sampling_rate)

dataset_elements = []
for el in dataset_hehe:
    dataset_elements.append(el)

In [73]:
input_values_list = []
attention_masks_list = []
labels_list = []
target_transcriptions_list = []

for feature, label, target_transcription in dataset_elements:      
    input_values_list.append(feature.input_values)
    attention_masks_list.append(feature.attention_mask)
    labels_list.append(label)
    target_transcriptions_list.append(target_transcription)

max_length_input = max(tensor.shape[1] for tensor in input_values_list)
max_length_mask = max(tensor.shape[1] for tensor in attention_masks_list)
max_length_label = max(tensor.shape[1] for tensor in labels_list)

padded_input_values_list = [torch.cat((tensor, torch.zeros(1, max_length_input - tensor.shape[1])), dim=1) for tensor in input_values_list]
padded_attention_masks_list = [torch.cat((tensor, torch.zeros(1, max_length_mask - tensor.shape[1])), dim=1) for tensor in attention_masks_list]
padded_labels_list = [torch.cat((tensor, torch.zeros(1, max_length_label - tensor.shape[1])), dim=1) for tensor in labels_list]

input_values_list = nn.utils.rnn.pad_sequence(padded_input_values_list, batch_first=True)
attention_masks_list = nn.utils.rnn.pad_sequence(padded_attention_masks_list, batch_first=True)    
labels_list = nn.utils.rnn.pad_sequence(padded_labels_list, batch_first=True)

print(input_values_list.shape)
print(labels_list.shape)

torch.Size([100, 1, 119809])
torch.Size([100, 1, 148])


In [12]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_values_list = []
    attention_masks_list = []
    labels_list = []
    target_transcriptions_list = []
    
    for feature, label, target_transcription in batch:      
        input_values_list.append(feature.input_values)
        attention_masks_list.append(feature.attention_mask)
        labels_list.append(label)
        target_transcriptions_list.append(target_transcription)

    max_length_input = max(tensor.shape[1] for tensor in input_values_list)
    max_length_mask = max(tensor.shape[1] for tensor in attention_masks_list)
    max_length_label = max(tensor.shape[1] for tensor in labels_list)

    padded_input_values_list = [torch.cat((tensor, torch.zeros(1, max_length_input - tensor.shape[1])), dim=1) for tensor in input_values_list]
    padded_attention_masks_list = [torch.cat((tensor, torch.zeros(1, max_length_mask - tensor.shape[1])), dim=1) for tensor in attention_masks_list]
    padded_labels_list = [torch.cat((tensor, torch.zeros(1, max_length_label - tensor.shape[1])), dim=1) for tensor in labels_list]

    input_values = nn.utils.rnn.pad_sequence(padded_input_values_list, batch_first=True).squeeze(1)
    attention_masks = nn.utils.rnn.pad_sequence(padded_attention_masks_list, batch_first=True).squeeze(1)    
    batch_labels = nn.utils.rnn.pad_sequence(padded_labels_list, batch_first=True).squeeze(1)
    
    batch_features = {
        'input_values': input_values,
        'attention_mask': attention_masks
    }
    
    return batch_features, batch_labels, target_transcriptions_list

## Original model evaluation

In [22]:
from datasets import load_metric

wer = load_metric('wer')
cer = load_metric('cer')

In [51]:
asr_model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [56]:
kwargs = {"num_workers": 1, "pin_memory": True}

test_loader = DataLoader(dataset=CustomDataset(test_set, processor, sampling_rate),
                            batch_size=10,
                            shuffle=False,
                            collate_fn=collate_fn,
                            **kwargs)

len(test_loader)

10

In [60]:
predictions_all = []
references_all = []

for batch in test_loader:
    features, labels, _ = batch   
    references = processor.batch_decode(labels)
    
    input_values = features["input_values"].to(device)
    attention_mask = features["attention_mask"].to(device)
    with torch.no_grad():
        logits = asr_model(input_values, attention_mask=attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predictions = processor.batch_decode(predicted_ids)

#     print('---')
#     print('Predictions:')
#     print(predictions)
#     print('References:')
#     print(references)
#     print('---')

    predictions_all.extend(predictions)
    references_all.extend(references)

In [61]:
wer_value = round(wer.compute(predictions=predictions_all, references=references_all), 4)
cer_value = round(cer.compute(predictions=predictions_all, references=references_all), 4)

print(f'WER: {wer_value} | CER: {cer_value}')

WER: 0.7101 | CER: 0.2478


## Original model with LM evaluation

In [70]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading /root/.cache/pyctcdecode/models--Yehor--wav2vec2-xls-r-base-uk-with-small-lm/snapshots/b958c7f8aecb6ad9de8702c2ec63462a3d02b62c/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [40]:
kwargs = {"num_workers": 1, "pin_memory": True}

test_loader = DataLoader(dataset=CustomDataset(test_set, processor, sampling_rate),
                            batch_size=10,
                            shuffle=False,
                            collate_fn=collate_fn,
                            **kwargs)

len(test_loader)

10

In [ ]:
predictions_all = []
references_all = []

for batch in test_loader:
    features, labels, references = batch   
    
    input_values = features["input_values"].to(device)
    attention_mask = features["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
        
    predictions = processor.batch_decode(logits.cpu().numpy()).text

#     print('---')
#     print('Predictions:')
#     print(predictions)
#     print('References:')
#     print(references)
#     print('---')

    predictions_all.extend(predictions)
    references_all.extend(references)

In [72]:
wer_value = round(wer.compute(predictions=predictions_all, references=references_all), 4)
cer_value = round(cer.compute(predictions=predictions_all, references=references_all), 4)

print(f'WER: {wer_value} | CER: {cer_value}')

WER: 0.7043 | CER: 0.2655


## Trainer

In [43]:
from datasets import load_metric
from tqdm.notebook import tqdm

class Trainer:
    def __init__(self, model, processor, train_dataloader: DataLoader, eval_dataloader: DataLoader, test_dataloader: DataLoader,
                 epochs=10, lr=1e-3):
        # logging, parameters, etc
        self.model = model
        self.processor = processor
        self.train_dataloader = train_dataloader
        self.eval_dataloader = eval_dataloader
        self.test_dataloader = test_dataloader
        self.epochs = epochs
        self.lr = lr
        self.criterion = nn.functional.ctc_loss
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.wer_func = load_metric("wer")

    def predict(self, batch):
        # generate prediction of the model (generate sequence) 
        features, _, _ = batch 
        input_values = features["input_values"].to(device)
        attention_mask = features["attention_mask"].to(device)
        
        with torch.no_grad():
            logits = self.model(input_values, attention_mask=attention_mask).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = self.processor.batch_decode(predicted_ids)
        
        return transcription

    def train_step(self, batch):
        # calculate loss on single batch
        features, labels, _ = batch 
        labels = labels.to(device)
        input_values = features["input_values"].to(device)
        attention_mask = features["attention_mask"].to(device)
        
        logits = self.model(input_values, attention_mask=attention_mask).logits
        log_probs = nn.functional.log_softmax(logits, dim=-1, dtype=torch.float32).transpose(0, 1)
#         print(input_values)

        labels_mask = labels >= 0
        target_lengths = labels_mask.sum(-1)
        flattened_targets = labels.masked_select(labels_mask)

        #  Computes the output length of the convolutional layers
        input_lengths = self.model._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)
        
        loss = self.criterion(log_probs, flattened_targets, input_lengths, target_lengths, reduction="mean", blank=36)
#         print(loss)
        
        return loss


    def train_epoch(self, epoch):
        # do backward prob, calculate WER on train epoch
        self.model.train()
        print("Training Epoch ", epoch)
        
        total_loss = 0
        predictions = []
        references = []
        
        for batch in tqdm(self.train_dataloader):
            self.optimizer.zero_grad()
            loss = self.train_step(batch)
            
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
            print(loss.item())
            
            _, _, reference = batch
            prediction = self.predict(batch)
            predictions.extend(prediction)
            references.extend(reference)
            
        total_loss /= len(self.train_dataloader)
        print(f"Epoch {epoch} | Train Loss: {total_loss:.4f}") 
        
        wer = self.wer_func.compute(predictions=predictions, references=references)
        print(f"Training WER: {wer:.4f}")
        
    def train(self):
        for e in range(self.epochs):
            self.train_epoch(e)
            self.validate()

    def validate(self):
        # calculate WER on validation subset
        self.model.eval()
        predictions = []
        references = []
        
        for batch in tqdm(self.eval_dataloader):
            _, _, reference = batch
            prediction = self.predict(batch)
            predictions.extend(prediction)
            references.extend(reference)
            
        wer = self.wer_func.compute(predictions=predictions, references=references)
        print(f"Validation WER: {wer:.4f}")

    def test(self):
        self.model.eval()
        predictions = []
        references = []
        
        for batch in tqdm(self.test_dataloader):
            _, _, reference = batch
            prediction = self.predict(batch)
            predictions.extend(prediction)
            references.extend(reference)
            
        wer = self.wer_func.compute(predictions=predictions, references=references)
        print(f"Test WER: {wer:.4f}")


In [49]:
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
model.config.ctc_zero_infinity = True
processor = Wav2Vec2Processor.from_pretrained(model_name)

batch_size = 2

train_loader = DataLoader(dataset=CustomDataset(train_set, processor, sampling_rate),
                            batch_size=batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            )
val_loader = DataLoader(dataset=CustomDataset(validation_set, processor, sampling_rate),
                            batch_size=batch_size,
                            shuffle=False,
                            collate_fn=collate_fn,
                            )
test_loader = DataLoader(dataset=CustomDataset(test_set, processor, sampling_rate),
                            batch_size=batch_size,
                            shuffle=False,
                            collate_fn=collate_fn,
                            )


Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [50]:
trainer = Trainer(model, processor, train_loader, val_loader, test_loader, epochs=3, lr=1e-2)
trainer.train()

Training Epoch  0


  0%|          | 0/40 [00:00<?, ?it/s]

2.9677224159240723
13.838479995727539
9.091469764709473
15.167634963989258
6.210369110107422
4.589627265930176
8.387992858886719
5.6813859939575195
5.976591110229492
3.6475107669830322
4.717825889587402
4.498201847076416
4.362736701965332
3.8385047912597656
3.8421554565429688
4.033626556396484
3.6633973121643066
3.880727529525757
3.7395682334899902
4.123683929443359
3.507385492324829
3.557971715927124
3.9132871627807617
3.2703704833984375
3.939302921295166
3.3298888206481934
3.428377628326416
3.7359490394592285
3.676765203475952
3.426372528076172
3.559164524078369
3.5540056228637695
3.0947813987731934
3.279843330383301
3.122380256652832
3.230301856994629
3.130767822265625
3.63541316986084
3.327981472015381
3.456728935241699
Epoch 0 | Train Loss: 4.6359
Training WER: 1.0000


  0%|          | 0/5 [00:00<?, ?it/s]

Validation WER: 1.0000
Training Epoch  1


...

Validation WER: 1.0000
Training Epoch  2


  0%|          | 0/40 [00:00<?, ?it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Epoch 2 | Train Loss: nan
Training WER: 1.0000


  0%|          | 0/5 [00:00<?, ?it/s]

Validation WER: 1.0000


In [51]:
trainer.test()

  0%|          | 0/5 [00:00<?, ?it/s]

Test WER: 1.0000


In [ ]:
# Train your own Language model
# go to https://github.com/kmario23/KenLM-training


for line in sys.stdin:
    for sentence in nltk.sent_tokenize(line):
        print(' '.join(nltk.word_tokenize(sentence)).lower())

In [ ]:
!bzcat *your_dataset.bz2* | python preprocess.py | ./kenlm/bin/lmplz -o 3 > bible.arpa

In [ ]:
# do we need to remove LM?
tokenizer = AutoTokenizer.from_pretrained(model_name)

vocab_dict = tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    path_to_ngram,
)

decoder.save_to_dir("wav2vec2_with_lm")